#Import Libraries

In [0]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from google.colab import drive

print("Module Loaded.")
print("TensorFlow Version :{}".format(tf.__version__))
print("NumPy Version :{}".format(np.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

TensorFlow 2.x selected.
Module Loaded.
TensorFlow Version :2.1.0-rc1
NumPy Version :1.17.4
Matplotlib Version :3.1.2


In [0]:
import os

drive.mount('/content/drive')
cwd = os.getcwd()
data_path = ('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture')

print(cwd)

!cp -r /content/drive/Shared\ drives/scsa_2019_b/Project_Ai-KEA/data_resize .

KeyboardInterrupt: ignored

In [0]:
class_name = np.array([item.name for item in data_path.glob('*')])
print(class_name)

> ### Data Resizing

In [0]:
# from PIL import Image

# for j in class_name[1:4]:
#     image = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data/furniture/'+j)
#     image = list(image.glob('*.jpg'))
#     for i in range(len(image)):
#         im = Image.open(str(image[i])).convert('RGB')
#         resize_image = im.resize((im.size[0]//2,im.size[1]//2))
#         resize_image.save('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture/{0}/{1}{2:03d}.jpg'.format(j,j,i))
#         print(j, i)

In [0]:
data_path = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture')

> ### Load Trained Model

In [0]:
# format : ai-kea-{day01}-{sey}
generator, _, _ = load_models("ai-kea",500)
show_images(0.5 * generator.predict(noise_data) + 0.5)

> ### Train Model

> ##### **Image Generator**

In [0]:
img_size = 112
batch_n = 256

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                #width_shift_range=0.3,
                                #height_shift_range=0.3,
                                #horizontal_flip=True, 
                                #vertical_flip=True,
                                #rotation_range=30,
                                #zoom_range=0.1,
                                #brightness_range=[0.8,1.2],
                                validation_split=0.2,
                                rescale=1./255)

In [0]:
train_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='training')
test_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='validation')

> ##### **Load VGG**

In [0]:
import tensorflow.keras.applications.vgg16 as vgg16

model_vgg = vgg16.VGG16(
    weights="imagenet", 
    include_top=False,    # dense layer 이 후는 제외
    input_tensor=tf.keras.layers.Input(shape=(img_size, img_size, 3))
    #input_tensor=keras.layers.Input(shape=(49, 49, 3))
)

model_vgg.summary()

> ##### **Fine Tuning**

> - Freeze Layers

In [0]:
# slicing which layers not to train
for layer in model_vgg.layers[:] :
  layer.trainable = False

for layer in model_vgg.layers :
  print(layer, layer.trainable)

> - Make Ai-KEA Model

In [0]:
ai-kea = tf.keras.models.Sequential(name='ai-kea')

ai-kea.add(model_vgg)

############### Here add new layers ###############

ai-kea.summary()

> ### Save Model

In [0]:
%%time

# format : ai-kea-{day01}-{sey}

if train_model == True:
  history15000 = train(15000, 500)
  save_models("ai-kea", 15000)
else:
  generator, _, _ = load_models("ai-kea",15000)

> ### Prediction

In [0]:
from keras.applications.vgg16 import decode_predictions
preds = model.predict(img)
# decode the results into a list of tuples (class, description, probability)
predict_result = decode_predictions(preds, top=2)
print('decode_predictions:\n ==>', predict_result)
print('Predicted:', predict_result[0][0][1])  # get descripsion

> ### Plot Result

In [0]:
import matplotlib.pyplot as plt

def show_images(generated_images, n=4, m=8, figsize=(9, 5)):
    fig = plt.figure(figsize=figsize)
    plt.subplots_adjust(top=1, bottom=0, hspace=0, wspace=0.05)
    for i in range(n):
        for j in range(m):
            k = i * m + j
            ax = fig.add_subplot(n, m, i * m + j + 1)
            ax.imshow(generated_images[k][:, :, 0])#, cmap=plt.cm.bone)
            ax.grid(False)
            ax.xaxis.set_ticks([])
            ax.yaxis.set_ticks([])
    plt.tight_layout()
    plt.show()

> ## CAM Model

### Define Model

In [0]:
# tf.keras.models.Model -> functional model
grad_model = tf.keras.models.Model(
    [model.inputs],                           # input: vgg16 inputs
    [model.get_layer('block5_conv3').output,  # outputs: vgg16 last conv
     model.output]                            # softmax out
)


NameError: ignored

**Build CAM**

In [0]:
# GradientTape 설정
with tf.GradientTape() as tape:
    conv_outputs, predict = grad_model(img)   # feature, predict
    class_out = predict[:, np.argmax(predict[0])]    # class     

# feature map
output = conv_outputs[0]

# Get gradient for input image
grads = tape.gradient(class_out, conv_outputs)[0]

# check grads' shape
grads.shape, output.shape

In [0]:
# Average gradients spatially
weights = tf.reduce_mean(grads, axis=(0, 1))

# Build a map of filters according to gradients importance
cam = np.ones(output.shape[0:2], dtype=np.float32)

for index, w in enumerate(weights):
    cam += w * output[:, :, index]

# check cam shape
cam.shape, weights.shape

> ### Blending CAM on original image

In [0]:
import cv2

# image read: ndarray type
img = cv2.imread(img_path)
# GBR -> RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # cv2는 기본적으로 image를 GBR로 처리

plt.imshow(img)

img.shape, type(img)

- CAM with HeatMap **(추후 Rectangle로 변경 필요)**

In [0]:
# resize cam
cam = cv2.resize(cam.numpy(), (img.shape[1], img.shape[0]))
cam = np.maximum(cam, 0)          # 0 이상의 값만 취함
# heatmap : minmax normalized one
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

# convert heatmap to rgb
heatmap = np.uint8(255 * heatmap)                       # RGB range (0~255)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # apply color map
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)      # colormap to RGB

# check heatmap shape
plt.imshow(heatmap)
heatmap.shape

- Blending HeatMap on Image

In [0]:
# blending heatmap: image: 100%, heatmap: 50%
output_image = cv2.addWeighted(img.astype('uint8'), 1,  # 1: 100%
                               heatmap, 0.5,            # 0.5: 50%
                               0)

In [0]:
# plot blended image
plt.imshow(output_image)
plt.axis('off')
plt.title(decode_predictions(preds, top=2)[0][0][1])